# Geração do modelo treinado

In [ ]:
from class_experiment import Experimento

num_neurons = 32
params = {
    'env_id': 'CartPole-v1',
    'policy_kwargs': dict(net_arch=[num_neurons, num_neurons]),
    'seeds': [0],
    'timesteps': int(1e4),
}

teste = Experimento(params)
teste.treinamento()

In [1]:
from class_experiment import Experimento
from GeraPlots import main_pipeline
import torch
import gc

num_neurons = 32
env_id = 'CartPole-v1'
params = {
    'policy_kwargs': dict(net_arch=[num_neurons, num_neurons]),
    'timesteps': int(1e6),
    'directory': f'../data/groups/{env_id}_{num_neurons}',
    'net_init': 2
}

enviroment_list = [
    # 'Acrobot-v1',
    'BipedalWalker-v3',
    # 'CarRacing-v3',
    # 'CartPole-v1',
    # 'MountainCar-v0',
    # 'Pendulum-v1',
    # 'MountainCarContinuous-v0',
    # 'LunarLander-v3'
]


for enviroment in enviroment_list:
    for init in range(10):
        params['directory'] = f'../data/groups/{enviroment}_{num_neurons}_{init}'
        for seed in range(20):
            params['seeds'] = [seed]
            params['env_id'] = enviroment
            params['net_init'] = init
            Ensaio = None  # Inicializa como None
            try:
                Ensaio = Experimento(params)
                Ensaio.treinamento()
                gc.collect()            
            finally:
                # Cleanup crítico
                if Ensaio is not None:
                    if hasattr(Ensaio, 'train_env'):
                        Ensaio.train_env.close()
                    if hasattr(Ensaio, 'model'):
                        # Limpa buffer do modelo
                        if hasattr(Ensaio.model, 'rollout_buffer'):
                            del Ensaio.model.rollout_buffer
                        del Ensaio.model
                    del Ensaio
                if torch.cuda.is_available():
                    torch.cuda.empty_cache()
                gc.collect()            
        main_pipeline(params['directory'])

DependencyNotInstalled: Box2D is not installed, you can install it by run `pip install swig` followed by `pip install "gymnasium[box2d]"`